# Realizar un Análisis de Varianza (ANOVA) en Python

Este tutorial explica cómo realizar un análisis de varianza (ANOVA) en Python, utilizando una base de datos en Excel. Se incluye la verificación de supuestos y, en caso de no cumplirse, cómo aplicar una alternativa no paramétrica.

## Pre-requisitos

Antes de comenzar, asegúrate de tener instaladas las siguientes bibliotecas en Python:

- pandas
- scipy
- openpyxl
- statsmodels

Puedes instalar estas bibliotecas usando pip:

```bash
pip install pandas scipy openpyxl statsmodels


## Paso 1: Cargar los Datos desde Excel

Carga los datos desde un archivo Excel. Asegúrate de que tu archivo tenga columnas separadas para cada grupo.

In [1]:
import pandas as pd

# Cargar datos desde Excel
archivo_excel = 'Anova.xlsx'
datos = pd.read_excel(archivo_excel)


## Paso 2: Verificar Supuestos

Antes de realizar un ANOVA, debes verificar ciertos supuestos:

### Supuesto de Normalidad

Comprueba la normalidad para cada grupo usando, por ejemplo, el test de Shapiro-Wilk.

In [2]:
from scipy.stats import shapiro

# Realizar el test de Shapiro-Wilk para cada grupo
for columna in datos.columns:
    stat, p = shapiro(datos[columna])
    print(f'{columna} - Shapiro-Wilk:', stat, p)

Lovastatina - Shapiro-Wilk: 0.868596076965332 0.09626547247171402
Atorvastatina - Shapiro-Wilk: 0.9281316995620728 0.4297696650028229
Rosuvastatina - Shapiro-Wilk: 0.7523679733276367 0.0038137026131153107


### Supuesto de Homogeneidad de Varianzas

Comprueba la homogeneidad de varianzas usando el test de Levene.

In [3]:
from scipy.stats import levene

stat, p = levene(*[datos[columna] for columna in datos.columns])
print('Levene:', stat, p)


Levene: 4.011403442832443 0.02983135624293939


## Paso 3: Realizar ANOVA

Si se cumplen los supuestos, realiza un ANOVA.

In [4]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Asegúrate de que tus datos están en formato largo para ANOVA
datos_largos = pd.melt(datos, var_name='grupos', value_name='valores')

# Ajustar el modelo
modelo = ols('valores ~ C(grupos)', data=datos_largos).fit()

# Realizar ANOVA
anova_resultados = anova_lm(modelo, typ=2)
print(anova_resultados)

# Para imprimir específicamente el valor de p, puedes hacer algo como esto:
valor_p = anova_resultados['PR(>F)'][0]
print(f'Valor de p: {valor_p}')

                 sum_sq    df          F        PR(>F)
C(grupos)  66888.866667   2.0  26.577106  4.172584e-07
Residual   33976.600000  27.0        NaN           NaN
Valor de p: 4.1725841571794765e-07


/var/folders/81/c9g9b_jj49s3fl45mph4gpg00000gn/T/ipykernel_70177/1159113934.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valor_p = anova_resultados['PR(>F)'][0]


## Pruebas Post Hoc para ANOVA y su Alternativa No Paramétrica

Después de realizar un ANOVA o su alternativa no paramétrica, si encuentras diferencias significativas, es importante realizar pruebas post hoc para identificar dónde están estas diferencias entre los grupos.

## Pruebas Post Hoc para ANOVA

Si el ANOVA revela diferencias significativas, utiliza pruebas post hoc como la prueba de Tukey.


In [5]:
# Realizar prueba post hoc de Tukey
from statsmodels.stats.multicomp import pairwise_tukeyhsd
resultado_tukey = pairwise_tukeyhsd(datos_largos['valores'], datos_largos['grupos'])
print(resultado_tukey)

         Multiple Comparison of Means - Tukey HSD, FWER=0.05         
    group1        group2    meandiff p-adj    lower    upper   reject
---------------------------------------------------------------------
Atorvastatina   Lovastatina    -28.4 0.1919  -67.7344  10.9344  False
Atorvastatina Rosuvastatina   -111.3    0.0 -150.6344 -71.9656   True
  Lovastatina Rosuvastatina    -82.9    0.0 -122.2344 -43.5656   True
---------------------------------------------------------------------


### Alternativa No Paramétrica: Prueba de Kruskal-Wallis

Si no se cumplen los supuestos, usa la prueba de Kruskal-Wallis como alternativa no paramétrica.

In [ ]:
from scipy.stats import kruskal

# Realizar prueba de Kruskal-Wallis
stat, p = kruskal(*[datos[columna] for columna in datos.columns])
print('Kruskal-Wallis:', stat, p)


### Pruebas Post Hoc para la Prueba de Kruskal-Wallis

Para la prueba de Kruskal-Wallis, puedes usar comparaciones por pares con ajustes de Bonferroni o Dunn.

#### Método de Bonferroni

In [ ]:
from statsmodels.sandbox.stats.multicomp import MultiComparison

# Realizar comparaciones múltiples con corrección de Bonferroni
mc = MultiComparison(datos_largos['valores'], datos_largos['grupos'])
resultado_bonferroni = mc.allpairtest(stats.ranksums, method='bonf')
print(resultado_bonferroni[0])


#### Prueba de Dunn

In [ ]:
from scikit_posthocs import posthoc_dunn

# Realizar la prueba de Dunn
resultado_dunn = posthoc_dunn(datos_largos, val_col='valores', group_col='grupos')
print(resultado_dunn)